# Whiskey Data Contract Quick Start

## Desired Process

Ideally each part of the process is as automated as much as possible.

1. **Commit 1:** Design and complete data contract collaboratively.
2. **Commit 2:** Register and share data contract in appropriate discoverable register.
3. **Commit 3:** Implement the schema interface and test.
4. **Commit 4:** Implement constraints on schema interface and test.
5. **Commit 5:** Implement business logic and test.
6. **Commit 6:** Release data product by granting access and publicising.


## Commit 1: Design and complete data contract collaboratively

## **Commit 2:** Register and share data contract in appropriate discoverable register.

## **Commit 3:** Implement the schema interface and test.

### Create schema which maps to data product

In [0]:
CREATE SCHEMA whiskey.inventory_v1;

### Create Models

In [0]:
CREATE OR REPLACE TABLE whiskey.inventory_v1.distilleries
(
  distillery_name STRING,
  distillery_country STRING,
  --last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
  CONSTRAINT distillery_name_pk PRIMARY KEY (distillery_name)
);

--ALTER TABLE whiskey.inventory_v1.distilleries SET TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'enabled');

In [0]:
DESCRIBE DETAIL whiskey.inventory_v1.distilleries;

In [0]:
CREATE OR REPLACE TABLE whiskey.inventory_v1.whiskeys
(
  whiskey_name STRING NOT NULL ,
  whiskey_type STRING NOT NULL,
  whiskey_age INT NOT NULL,
  distillery_name STRING NOT NULL,
  distillery_country STRING NOT NULL,
  --last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
  CONSTRAINT distillery_name_fk FOREIGN KEY(distillery_name) REFERENCES whiskey.inventory_v1.distilleries
)

In [0]:
DESCRIBE DETAIL whiskey.inventory_v1.whiskeys;

### Test

## **Commit 4:** Implement constraints on schema interface and test.

### Constraints Model 1

In [0]:
ALTER TABLE whiskey.inventory_v1.distilleries ADD CONSTRAINT distillery_name_chk CHECK (distillery_name IN ('Glenmorangie','Penderyn'));
ALTER TABLE whiskey.inventory_v1.distilleries ADD CONSTRAINT distillery_country_chk CHECK (distillery_country IN ('Scotland','Wales'));

### Constraints Model 2

In [0]:
ALTER TABLE whiskey.inventory_v1.whiskeys ADD CONSTRAINT whiskey_name_chk CHECK (whiskey_name IN ('Myth','Glenmorangie 12'));
ALTER TABLE whiskey.inventory_v1.whiskeys ADD CONSTRAINT whiskey_type_chk CHECK (whiskey_type IN ('Single Malt','Blended'));
ALTER TABLE whiskey.inventory_v1.whiskeys ADD CONSTRAINT whiskey_age_chk CHECK (whiskey_age BETWEEN 3 AND 100);

ALTER TABLE whiskey.inventory_v1.whiskeys ADD CONSTRAINT distillery_name_chk CHECK (distillery_name IN ('Glenmorangie','Penderyn'));
ALTER TABLE whiskey.inventory_v1.whiskeys ADD CONSTRAINT distillery_country_chk CHECK (distillery_name IN ('Scotland','Wales'));

### Test

In [0]:
%python
def try_insert(sql_statement):
    try:
        spark.sql(sql_statement)
        print("Insert succeeded.")
    except:
        print("Insert failed.")

# Run inserts separately
try_insert("""
    INSERT INTO whiskey.inventory_v1.distilleries (distillery_name, distillery_country)
    VALUES ('Glenmorangie', 'Scotland')
""")

try_insert("""
    INSERT INTO whiskey.inventory_v1.distilleries (distillery_name, distillery_country)
    VALUES ('Penderyn', 'Wales')
""")

try_insert("""
    INSERT INTO whiskey.inventory_v1.distilleries (distillery_name, distillery_country)
    VALUES ('Guff', 'Guff')  -- known to violate constraints
""")


In [0]:
%python
def try_insert(sql_statement):
    try:
        spark.sql(sql_statement)
        print("Insert succeeded.")
    except:
        print("Insert failed.")

# Run inserts separately
try_insert("""
    INSERT INTO whiskey.inventory_v1.whiskeys (whiskey_name, whiskey_age, whiskey_type, distillery_name, distillery_country)
    VALUES ('Glenmorangie 12', 12, 'Single Malt', 'Glenmorangie', 'Scotland')
""")

try_insert("""
    INSERT INTO whiskey.inventory_v1.whiskeys (whiskey_name, whiskey_age, whiskey_type, distillery_name, distillery_country)
    VALUES ('Myth', 5, 'Single Malt', 'Penderyn', 'Wales')
""")

try_insert("""
    INSERT INTO whiskey.inventory_v1.whiskeys (whiskey_name, whiskey_age, whiskey_type, distillery_name, distillery_country)
    VALUES ('Invalid Whiskey', 10, 'Blended', 'Nonexistent Distillery', 'Nowhere')
""")



## Gotchas

1. Primary key and foreign key constraints are informational only.

## Wishlist

1. Isolate development environment such as a container.
2. Declaratively publish schema interface (pulumi?).
2. Ensure domain driven design carried out to discover domains and services.
3. Ensure that the data product mapped to the schema is geared towards a value and not an entity.
4. Cross field validation checks e.g. Penderyn only in Wales - unit test?
5. Modularise constraint testing if repeated across models.
6. Low resourced prevention of duplicates.
7. Action commands to interface with schema akin to POST, READ etc so goes through this logic to interact with data products.
8. Prevent all reads other than via correct POST action - does this mean expose views only for READ and have appropriate functions for POST?
10. Default ability before create table.